# Reproducibility - resume

1. Create and commit two versions of dataframe -> v1, v2
2. Run flow 1 with v1 data -> crashes b/c flow 2 hasn't run
3. Run flow 2 with v2 data
4. Resume flow 1 successfully

In [19]:
import pandas as pd
from doltpy.core import Dolt
from doltpy.core.write import import_df

dolt = Dolt.init("foo")

df_v1 = pd.DataFrame({"A": [1,1,1], "B": [1,1,1]})
df_v2 = pd.DataFrame({"A": [1,1,1,2,2,2], "B": [1,1,1,2,2,2]})

import_df(dolt, "bar", df_v1.reset_index(), ["index"], "create")
dolt.add("bar")
dolt.commit("Initialize bar")

v1 = list(dolt.log(number="1").keys())[0]

import_df(dolt, "bar", df_v2.reset_index(), ["index"], "update")
dolt.add("bar")
dolt.commit("Add rows to bar; initalize baz")

v2 = list(dolt.log(number="1").keys())[0]

01-15 15:28:22 doltpy.core.dolt INFO     Creating directory foo
01-15 15:28:22 doltpy.core.dolt INFO     Creating a new repo in foo
01-15 15:28:22 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
01-15 15:28:22 doltpy.core.write.write INFO     Importing to table bar in dolt directory located in foo, import mode create
01-15 15:28:22 doltpy.core.dolt INFO     Rows Processed: 3, Additions: 3, Modifications: 0, Had No Effect: 0
01-15 15:28:22 doltpy.core.dolt INFO     
01-15 15:28:23 doltpy.core.dolt INFO     commit vcvdem6noni0sgl6erk06s4r2gtj2i2q
Author: Max Hoffman <maximilian.wolfgang1@gmail.com>
Date:   Fri Jan 15 15:28:23 -0800 2021

	Initialize bar


01-15 15:28:23 doltpy.core.write.write INFO     Importing to table bar in dolt directory located in foo, import mode update
01-15 15:28:23 doltpy.core.dolt INFO     Rows Processed: 6, Additions: 3, Modifications: 0, Had No Effect: 3
01-15 15:28:23 doltpy.core.dolt INFO     
01-15 15:28:23

In [35]:
!cat succeeds_second.py

import pickle

from metaflow import FlowSpec, step, DoltDT, Parameter, Flow
import pandas as pd
from sklearn import tree

class SucceedsSecondDemo(FlowSpec):

    bar_version = Parameter('bar-version',  help="Specifc the tag for the input version", required=True)

    @step
    def start(self):
        with DoltDT(run=self, doltdb_path='foo', branch=self.bar_version) as dolt:
            self.df = dolt.read_table('bar')
        first = Flow("SucceedsFirstDemo").latest_successful_run

        self.next(self.middle)

    @step
    def middle(self):
        with DoltDT(run=self, doltdb_path='foo', branch=self.bar_version) as dolt:
            df = self.df
            df["B"] = df["B"].map(lambda x: x*2)

            dolt.write_table(table_name='baz', df=df, pks=['index'])

        self.next(self.end)

    @step
    def end(self):
        pass


if __name__ == '__main__':
    SucceedsSecondDemo()


In [36]:
!cat succeeds_first.py

import pickle

from metaflow import FlowSpec, step, DoltDT, Parameter
import pandas as pd
from sklearn import tree

class SucceedsFirstDemo(FlowSpec):

    bar_version = Parameter('bar-version',  help="Specifc the tag for the input version", required=True)

    @step
    def start(self):
        with DoltDT(run=self, doltdb_path='foo', branch=self.bar_version) as dolt:
            self.df = dolt.read_table('bar')

        self.next(self.middle)

    @step
    def middle(self):
        with DoltDT(run=self, doltdb_path='foo', branch=self.bar_version) as dolt:

            df = self.df
            df["B"] = df["B"].map(lambda x: x*2)

            dolt.write_table(table_name='baz', df=df, pks=['index'])

        self.next(self.end)

    @step
    def end(self):
        pass


if __name__ == '__main__':
    SucceedsFirstDemo()


In [30]:
!poetry run python3 succeeds_second.py run --bar-version $v1

Metaflow 2.2.5.post19+gitd26b8cd executing SucceedsSecondDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-15 15:36:49.777 Workflow starting (run-id 1610753809771762):
2021-01-15 15:36:49.784 [1610753809771762/start/1 (pid 11986)] Task is starting.
2021-01-15 15:36:50.869 [1610753809771762/start/1 (pid 11986)] 01-15 15:36:50 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-15 15:36:50.958 [1610753809771762/start/1 (pid 11986)] 01-15 15:36:50 doltpy.core.dolt INFO     * master                                        	r44hf3g77op6cfhmhj10fpa59qu94p00
2021-01-15 15:36:51.080 [1610753809771762/start/1 (pid 11986)]   tmp_7496997185783575499                       	vcvdem6noni0sgl6erk06s4r2gtj2i2q
2021-01-15 15:36:51.080 [1610753809771762/start/1 (pid 11986)] 
2021-01-15 15:36:51.080 [1610753809771762/start/1 (pid 11986)] 01-15 15:36:51 doltpy.core.dolt INFO     Swit

In [31]:
!poetry run python3 succeeds_first.py run --bar-version $v2

Metaflow 2.2.5.post19+gitd26b8cd executing SucceedsFirstDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-15 15:37:48.172 Workflow starting (run-id 1610753868160529):
2021-01-15 15:37:48.179 [1610753868160529/start/1 (pid 12023)] Task is starting.
2021-01-15 15:37:49.409 [1610753868160529/start/1 (pid 12023)] 01-15 15:37:49 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-15 15:37:49.508 [1610753868160529/start/1 (pid 12023)] 01-15 15:37:49 doltpy.core.dolt INFO       master                                        	r44hf3g77op6cfhmhj10fpa59qu94p00
2021-01-15 15:37:49.651 [1610753868160529/start/1 (pid 12023)] * tmp_65359573685889210212                      	vcvdem6noni0sgl6erk06s4r2gtj2i2q
2021-01-15 15:37:49.651 [1610753868160529/start/1 (pid 12023)]   tmp_7496997185783575499                       	vcvdem6noni0sgl6erk06s4r2gtj2i2q
2021-01-15 15:37:49.651 [161

In [34]:
!poetry run python3 succeeds_second.py resume start

Metaflow 2.2.5.post19+gitd26b8cd executing SucceedsSecondDemo for user:max-hoffman
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
2021-01-15 15:38:34.003 Gathering required information to resume run (this may take a bit of time)...
2021-01-15 15:38:34.011 Workflow starting (run-id 1610753914002543):
2021-01-15 15:38:34.018 [1610753914002543/start/1 (pid 12116)] Task is starting.
2021-01-15 15:38:35.150 [1610753914002543/start/1 (pid 12116)] 01-15 15:38:35 doltpy.core.dolt INFO     Creating engine for Dolt SQL Server instance running on 127.0.0.1:3306
2021-01-15 15:38:35.240 [1610753914002543/start/1 (pid 12116)] 01-15 15:38:35 doltpy.core.dolt INFO       master                                        	r44hf3g77op6cfhmhj10fpa59qu94p00
2021-01-15 15:38:35.365 [1610753914002543/start/1 (pid 12116)] * tmp_4823254690243868365                       	0cbd00l60kvopvd3qmcohm8ej44pdp2b
2021-01-15 15:38:35.365 [1610753914002543/start/1 (pid 12116)]   tmp_5

2021-01-15 15:38:39.040 [1610753914002543/end/3 (pid 12181)] 01-15 15:38:39 doltpy.core.system_helpers INFO     No processes to clean up, exiting
2021-01-15 15:38:39.202 [1610753914002543/end/3 (pid 12181)] Task finished successfully.
2021-01-15 15:38:39.203 Done!
01-15 15:38:39 doltpy.core.system_helpers INFO     Before exiting cleaning up child processes
01-15 15:38:39 doltpy.core.system_helpers INFO     No processes to clean up, exiting
